In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime

# from finrl.config import config
from finrl.neo_finrl.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.neo_finrl.env_portfolio_allocation.portfolio_allocation import StockPortfolioEnv

from finrl.drl_agents.stablebaselines3.models import DRLAgent
# from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts

import sys

sys.path.append("..")

In [2]:
df = pd.read_csv('usdt_1h_ada_algo_btc_eth_tur.csv')
df['date'] = pd.to_datetime(df['date'])
start_time = df['date'][0]
end_time = df['date'][len(df) - 1]
technical_indicators = ['boll_ub',
 'boll_lb',
 'rsi_56',
 'rsi_336',
 'rsi_2352',
 'cci_56',
 'dx_56',
 'close_4_sma',
 'close_72_sma',
 'close_48_ema',
 'close_104_ema',
 'close_36_ema']

In [3]:
df = df[['date', 'open', 'high', 'low', 'close', 'volume', 'tic']]

# df.loc[df['tic'] == 'BTC_USDT', 'open'] /= 10000
# df.loc[df['tic'] == 'BTC_USDT', 'close'] /= 10000
# df.loc[df['tic'] == 'BTC_USDT', 'high'] /= 10000
# df.loc[df['tic'] == 'BTC_USDT', 'low'] /= 10000
# df.loc[df['tic'] == 'BTC_USDT', 'volume'] *= 10000

# df.loc[df['tic'] == 'ETH_USDT', 'open'] /= 1000
# df.loc[df['tic'] == 'ETH_USDT', 'close'] /= 1000
# df.loc[df['tic'] == 'ETH_USDT', 'high'] /= 1000
# df.loc[df['tic'] == 'ETH_USDT', 'low'] /= 1000
# df.loc[df['tic'] == 'ETH_USDT', 'volume'] *= 1000

# df.loc[df['tic'] == 'ADA_USDT', 'open'] /= 10
# df.loc[df['tic'] == 'ADA_USDT', 'close'] /= 10
# df.loc[df['tic'] == 'ADA_USDT', 'high'] /= 10
# df.loc[df['tic'] == 'ADA_USDT', 'low'] /= 10
# df.loc[df['tic'] == 'ADA_USDT', 'volume'] *= 10

In [4]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    tech_indicator_list=technical_indicators,
                    user_defined_feature = False)

df = fe.preprocess_data(df)

Successfully added technical indicators


In [5]:
# df.to_csv('/content/drive/MyDrive/data/portfolio_df.csv')

In [6]:
# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
# look back is one year
lookback=720*3
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  covs = return_lookback.cov().values 
  cov_list.append(covs)
  
df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)
df.head()

,date,open,high,low,close,volume,tic,boll_ub,boll_lb,rsi_56,rsi_336,rsi_2352,cci_56,dx_56,close_4_sma,close_72_sma,close_48_ema,close_104_ema,close_36_ema,cov_list
0,2019-09-20 15:00:00,0.05159,0.05205,0.05155,0.05173,7.368821e+06,ADA_USDT,0.053810,0.049722,53.379375,51.880940,48.377046,1.087486,2.750886,0.051348,0.051148,0.051313,0.050154,0.051489,"[[0.00010444166579769002, 5.438065070131934e-0..."
1,2019-09-20 15:00:00,0.31900,0.32100,0.31900,0.32000,7.070001e+04,ALGO_USDT,0.324878,0.315622,48.020831,46.555533,45.433072,-50.938807,4.141416,0.319500,0.325819,0.322747,0.325008,0.321843,"[[0.00010444166579769002, 5.438065070131934e-0..."
2,2019-09-20 15:00:00,10157.25000,10189.50000,10125.00000,10140.95000,1.507407e+03,BTC_USDT,10373.562847,9859.947153,49.637721,49.441086,49.852803,27.281361,0.403765,10157.007500,10125.886944,10111.830484,10147.448714,10110.378008,"[[0.00010444166579769002, 5.438065070131934e-0..."
3,2019-09-20 15:00:00,217.34000,218.24000,216.24000,216.80000,2.097448e+04,ETH_USDT,224.611788,210.138212,58.540904,54.512479,49.055765,72.786074,11.699417,217.195000,211.357222,213.098205,206.738928,214.509011,"[[0.00010444166579769002, 5.438065070131934e-0..."
4,2019-09-20 16:00:00,0.05173,0.05204,0.05117,0.05185,1.484533e+07,ADA_USDT,0.053817,0.049808,53.599567,51.939663,48.388529,-6.636946,0.124364,0.051683,0.051208,0.051335,0.050186,0.051508,"[[0.00010419746593874823, 5.4345651233928975e-..."


In [7]:
df['cov_list'].shape

(67252,)

In [8]:
end_train = '2021-01-31 23:00:00'
train = data_split(df, start_time, end_train)
trade = data_split(df, end_train, end_time)
print(len(train))
print(len(trade))

47852
19396


In [9]:
train.head()

,date,open,high,low,close,volume,tic,boll_ub,boll_lb,rsi_56,rsi_336,rsi_2352,cci_56,dx_56,close_4_sma,close_72_sma,close_48_ema,close_104_ema,close_36_ema,cov_list
0,2019-09-20 15:00:00,0.05159,0.05205,0.05155,0.05173,7.368821e+06,ADA_USDT,0.053810,0.049722,53.379375,51.880940,48.377046,1.087486,2.750886,0.051348,0.051148,0.051313,0.050154,0.051489,"[[0.00010444166579769002, 5.438065070131934e-0..."
0,2019-09-20 15:00:00,0.31900,0.32100,0.31900,0.32000,7.070001e+04,ALGO_USDT,0.324878,0.315622,48.020831,46.555533,45.433072,-50.938807,4.141416,0.319500,0.325819,0.322747,0.325008,0.321843,"[[0.00010444166579769002, 5.438065070131934e-0..."
0,2019-09-20 15:00:00,10157.25000,10189.50000,10125.00000,10140.95000,1.507407e+03,BTC_USDT,10373.562847,9859.947153,49.637721,49.441086,49.852803,27.281361,0.403765,10157.007500,10125.886944,10111.830484,10147.448714,10110.378008,"[[0.00010444166579769002, 5.438065070131934e-0..."
0,2019-09-20 15:00:00,217.34000,218.24000,216.24000,216.80000,2.097448e+04,ETH_USDT,224.611788,210.138212,58.540904,54.512479,49.055765,72.786074,11.699417,217.195000,211.357222,213.098205,206.738928,214.509011,"[[0.00010444166579769002, 5.438065070131934e-0..."
1,2019-09-20 16:00:00,0.05173,0.05204,0.05117,0.05185,1.484533e+07,ADA_USDT,0.053817,0.049808,53.599567,51.939663,48.388529,-6.636946,0.124364,0.051683,0.051208,0.051335,0.050186,0.051508,"[[0.00010419746593874823, 5.4345651233928975e-..."


In [10]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 4, State Space: 4


In [11]:
env_kwargs = {
    "initial_amount": 100000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": technical_indicators, 
    "action_space": stock_dimension + 1, 
    "reward_scaling": 1e-2,
    "min_trans_amount": [1, 0, 5, 4]
}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

In [12]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [13]:
# import os
# if not os.path.exists("./" + config.DATA_SAVE_DIR):
#     os.makedirs("./" + config.DATA_SAVE_DIR)
# if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
#     os.makedirs("./" + config.TRAINED_MODEL_DIR)
# if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
#     os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
# if not os.path.exists("./" + config.RESULTS_DIR):
#     os.makedirs("./" + config.RESULTS_DIR)


In [14]:
# initialize
agent = DRLAgent(env = env_train)

a2c_params_tuning = {'n_steps':5, 
                     'ent_coef':0.005, 
                     'learning_rate':0.0003,
}
model_a2c = agent.get_model("a2c",model_kwargs=a2c_params_tuning)
trained_a2c = agent.train_model(model= model_a2c, tb_log_name= "a2c", total_timesteps= 60000)


{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0003}
Using cpu device
Logging to tensorboard_log/a2c/a2c_15
-------------------------------------
| time/                 |           |
|    fps                | 262       |
|    iterations         | 100       |
|    time_elapsed       | 1         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -7.1      |
|    explained_variance | 0         |
|    learning_rate      | 0.0003    |
|    n_updates          | 99        |
|    policy_loss        | -39       |
|    reward             | 6.8329268 |
|    std                | 1         |
|    value_loss         | 40.9      |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 287       |
|    iterations         | 200       |
|    time_elapsed       | 3         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy

-------------------------------------
| time/                 |           |
|    fps                | 357       |
|    iterations         | 1400      |
|    time_elapsed       | 19        |
|    total_timesteps    | 7000      |
| train/                |           |
|    entropy_loss       | -7.12     |
|    explained_variance | 0         |
|    learning_rate      | 0.0003    |
|    n_updates          | 1399      |
|    policy_loss        | 2.75      |
|    reward             | 0.6414327 |
|    std                | 1         |
|    value_loss         | 0.352     |
-------------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 358         |
|    iterations         | 1500        |
|    time_elapsed       | 20          |
|    total_timesteps    | 7500        |
| train/                |             |
|    entropy_loss       | -7.12       |
|    explained_variance | 0           |
|    learning_rate      | 0.0003

-------------------------------------
| time/                 |           |
|    fps                | 346       |
|    iterations         | 2700      |
|    time_elapsed       | 38        |
|    total_timesteps    | 13500     |
| train/                |           |
|    entropy_loss       | -7.1      |
|    explained_variance | 1.19e-07  |
|    learning_rate      | 0.0003    |
|    n_updates          | 2699      |
|    policy_loss        | -2.9      |
|    reward             | 0.6726446 |
|    std                | 1         |
|    value_loss         | 1.92      |
-------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 347        |
|    iterations         | 2800       |
|    time_elapsed       | 40         |
|    total_timesteps    | 14000      |
| train/                |            |
|    entropy_loss       | -7.1       |
|    explained_variance | 0          |
|    learning_rate      | 0.0003     |
| 

-------------------------------------
| time/                 |           |
|    fps                | 350       |
|    iterations         | 4100      |
|    time_elapsed       | 58        |
|    total_timesteps    | 20500     |
| train/                |           |
|    entropy_loss       | -7.14     |
|    explained_variance | 0         |
|    learning_rate      | 0.0003    |
|    n_updates          | 4099      |
|    policy_loss        | 21.1      |
|    reward             | 1.2813807 |
|    std                | 1.01      |
|    value_loss         | 11.1      |
-------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 350        |
|    iterations         | 4200       |
|    time_elapsed       | 59         |
|    total_timesteps    | 21000      |
| train/                |            |
|    entropy_loss       | -7.14      |
|    explained_variance | -1.19e-07  |
|    learning_rate      | 0.0003     |
| 

--------------------------------------
| time/                 |            |
|    fps                | 353        |
|    iterations         | 5400       |
|    time_elapsed       | 76         |
|    total_timesteps    | 27000      |
| train/                |            |
|    entropy_loss       | -7.19      |
|    explained_variance | 0          |
|    learning_rate      | 0.0003     |
|    n_updates          | 5399       |
|    policy_loss        | 10.2       |
|    reward             | 0.79568166 |
|    std                | 1.02       |
|    value_loss         | 3.63       |
--------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 353        |
|    iterations         | 5500       |
|    time_elapsed       | 77         |
|    total_timesteps    | 27500      |
| train/                |            |
|    entropy_loss       | -7.19      |
|    explained_variance | 0          |
|    learning_rate      |

--------------------------------------
| time/                 |            |
|    fps                | 351        |
|    iterations         | 6800       |
|    time_elapsed       | 96         |
|    total_timesteps    | 34000      |
| train/                |            |
|    entropy_loss       | -7.21      |
|    explained_variance | 0          |
|    learning_rate      | 0.0003     |
|    n_updates          | 6799       |
|    policy_loss        | 22.5       |
|    reward             | 0.24413963 |
|    std                | 1.02       |
|    value_loss         | 14.5       |
--------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 349        |
|    iterations         | 6900       |
|    time_elapsed       | 98         |
|    total_timesteps    | 34500      |
| train/                |            |
|    entropy_loss       | -7.21      |
|    explained_variance | -1.19e-07  |
|    learning_rate      |

--------------------------------------
| time/                 |            |
|    fps                | 351        |
|    iterations         | 8200       |
|    time_elapsed       | 116        |
|    total_timesteps    | 41000      |
| train/                |            |
|    entropy_loss       | -7.21      |
|    explained_variance | 0          |
|    learning_rate      | 0.0003     |
|    n_updates          | 8199       |
|    policy_loss        | -29.3      |
|    reward             | -0.2780836 |
|    std                | 1.02       |
|    value_loss         | 25         |
--------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 352       |
|    iterations         | 8300      |
|    time_elapsed       | 117       |
|    total_timesteps    | 41500     |
| train/                |           |
|    entropy_loss       | -7.21     |
|    explained_variance | 0         |
|    learning_rate      | 0.0003  

begin_total_asset:100000
end_total_asset:100000
Sharpe:  nan
------------------------------------
| time/                 |          |
|    fps                | 354      |
|    iterations         | 9600     |
|    time_elapsed       | 135      |
|    total_timesteps    | 48000    |
| train/                |          |
|    entropy_loss       | -7.2     |
|    explained_variance | 0        |
|    learning_rate      | 0.0003   |
|    n_updates          | 9599     |
|    policy_loss        | -33.1    |
|    reward             | 5.743412 |
|    std                | 1.02     |
|    value_loss         | 20.5     |
------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 354       |
|    iterations         | 9700      |
|    time_elapsed       | 136       |
|    total_timesteps    | 48500     |
| train/                |           |
|    entropy_loss       | -7.2      |
|    explained_variance | 0         |
|    

---------------------------------------
| time/                 |             |
|    fps                | 355         |
|    iterations         | 11000       |
|    time_elapsed       | 154         |
|    total_timesteps    | 55000       |
| train/                |             |
|    entropy_loss       | -7.2        |
|    explained_variance | -1.19e-07   |
|    learning_rate      | 0.0003      |
|    n_updates          | 10999       |
|    policy_loss        | 10.5        |
|    reward             | -0.26535556 |
|    std                | 1.02        |
|    value_loss         | 4.21        |
---------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 354        |
|    iterations         | 11100      |
|    time_elapsed       | 156        |
|    total_timesteps    | 55500      |
| train/                |            |
|    entropy_loss       | -7.22      |
|    explained_variance | 0          |
|    lear

In [15]:
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)

In [16]:
df_daily_return, df_actions, _, _ = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)

begin_total_asset:100000
end_total_asset:100000
Sharpe:  nan
hit end!


In [17]:
df_daily_return

,date,account_value
0,2021-01-31 23:00:00,100000.000000
1,2021-02-01 00:00:00,98030.823500
2,2021-02-01 01:00:00,97658.003207
3,2021-02-01 02:00:00,99410.129834
4,2021-02-01 03:00:00,98609.292310
...,...,...
4844,2021-08-22 06:00:00,100602.712990
4845,2021-08-22 07:00:00,101932.635280
4846,2021-08-22 08:00:00,102844.626648
4847,2021-08-22 09:00:00,102506.005127


In [18]:
df_actions

,cash,ADA_USDT,ALGO_USDT,BTC_USDT,ETH_USDT,cashh,ada,algo,btc,eth
date,,,,,,,,,,
2021-01-31 23:00:00,100000.000000,0.0,0.0,0.00000,0.0000,1.000000,0.000000,0.000000,0.000000,0.000000
2021-02-01 00:00:00,18822.250306,36503.0,52889.0,0.65080,9.6386,0.189033,0.126509,0.343886,0.214063,0.126509
2021-02-01 01:00:00,28855.664980,31138.3,45840.0,0.59008,8.2321,0.294455,0.108324,0.294455,0.194441,0.108324
2021-02-01 02:00:00,31453.113506,34193.6,50595.0,0.35671,8.9855,0.322202,0.118532,0.322202,0.118532,0.118532
2021-02-01 03:00:00,22703.798050,65722.8,45999.0,0.33417,10.2251,0.228548,0.227131,0.299801,0.110291,0.134230
...,...,...,...,...,...,...,...,...,...,...
2021-08-22 06:00:00,32681.748636,4894.3,28024.0,0.24369,3.6968,0.322202,0.118532,0.322202,0.118532,0.118532
2021-08-22 07:00:00,32414.175878,4898.9,28215.0,0.24424,3.6993,0.322202,0.118532,0.322202,0.118532,0.118532
2021-08-22 08:00:00,10091.011501,10156.0,23170.0,0.56214,3.6357,0.099337,0.244883,0.270026,0.270026,0.115727


In [19]:
closed = np.append([1] , trade.loc[0].close.values.tolist())
closed 

array([1.000000e+00, 3.465700e-01, 6.502000e-01, 3.289186e+04,
       1.312510e+03])

In [27]:
test_df = df_actions[['cashh', 'ada', 'algo', 'btc', 'eth']] 
test_df.iloc[1] * 100000

cashh    18903.346360
ada      12650.869787
algo     34388.631582
btc      21406.279504
eth      12650.869787
Name: 2021-02-01 00:00:00, dtype: float64

In [20]:
trade.head()

,date,open,high,low,close,volume,tic,boll_ub,boll_lb,rsi_56,rsi_336,rsi_2352,cci_56,dx_56,close_4_sma,close_72_sma,close_48_ema,close_104_ema,close_36_ema,cov_list
0,2021-01-31 23:00:00,0.34419,0.34680,0.34000,0.34657,1.822425e+07,ADA_USDT,0.372252,0.343087,49.434094,51.319792,52.004164,-83.329619,3.873874,0.347712,0.352132,0.353008,0.349065,0.353935,"[[0.0002662892356814467, 0.0001679717184276767..."
0,2021-01-31 23:00:00,0.65460,0.66840,0.64970,0.65020,1.600619e+06,ALGO_USDT,0.678525,0.637315,53.494131,52.660269,51.259631,43.387665,26.935413,0.665275,0.620978,0.642627,0.616427,0.649815,"[[0.0002662892356814467, 0.0001679717184276767..."
0,2021-01-31 23:00:00,32862.65000,33073.55000,32555.00000,32891.86000,3.707195e+03,BTC_USDT,34704.267635,32828.498365,48.134553,50.014009,51.890975,-128.409564,19.160449,33194.537500,34196.435000,33820.201227,33541.109954,33808.451218,"[[0.0002662892356814467, 0.0001679717184276767..."
0,2021-01-31 23:00:00,1311.01000,1320.00000,1296.29000,1312.51000,6.838198e+04,ETH_USDT,1393.303434,1313.646566,47.634665,51.371315,52.426816,-202.205100,23.240875,1325.952500,1355.960556,1351.097920,1342.764507,1351.125262,"[[0.0002662892356814467, 0.0001679717184276767..."
1,2021-02-01 00:00:00,0.34655,0.34758,0.33961,0.34103,1.451201e+07,ADA_USDT,0.372582,0.340482,48.060355,51.099948,51.939022,-95.049684,2.771485,0.344775,0.352113,0.352519,0.348912,0.353237,"[[0.0002660495491612565, 0.0001681075604836601..."


In [ ]:
def calculate_sharpe(df):
  df['daily_return'] = df['account_value'].pct_change(1)
  if df['daily_return'].std() !=0:
    sharpe = df['daily_return'].mean() / df['daily_return'].std()
    return sharpe
  else:
    return 0

In [ ]:
from IPython.display import clear_output
import torch as th

def objective_ddpg(ddpg_params):
  print(ddpg_params)
  e_train_gym = StockPortfolioEnv(df= train, **env_kwargs)
  env_train, _ = e_train_gym.get_sb_env()
  agent = DRLAgent(env = env_train)
  
  net_params = dict(activation_fn=th.nn.Tanh, net_arch=list(ddpg_params['net']))
  ddpg_params.pop('net')
  ddpg_params['buffer_size'] = int(ddpg_params['buffer_size'])
  ddpg_params['batch_size'] = int(ddpg_params['batch_size'])

  model_ddpg = agent.get_model("ddpg",model_kwargs= ddpg_params, policy_kwargs= net_params)
  trained_ddpg = agent.train_model(model= model_ddpg, tb_log_name= "ddpg", total_timesteps= 60000)
  clear_output(wait= True)
  
  e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)
  df_account_value, df_actions, _, _ = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym)
  if -calculate_sharpe(df_account_value) < best_neg_sharpe:
    model_ddpg.save("/content/drive/MyDrive/data/portfolio_model_ddpg")
  
  return {
      'loss': -calculate_sharpe(df_account_value),
  }

In [ ]:
net_archs = [[64,64], [96, 96], [128, 128], [64, 128, 128], [96, 128, 128]]

In [ ]:
import json

best_neg_sharpe = -0.02
best_ddpg = {}

for trial in range(30):
  net = np.random.randint(0, 5)
  buffer_size = np.random.randint(10000, 20000)
  batch_size = np.random.randint(100, 200)
  learning_rate = np.random.uniform(0.0001, 0.007)
  gamma = np.random.uniform(0.97, 0.99)
  params = {
      "net":net_archs[net],
      "buffer_size":buffer_size,
      "batch_size":batch_size,
      "learning_rate":learning_rate,
      "gamma":gamma
  }
  res = objective_ddpg(params)
  if res['loss'] < best_neg_sharpe:
    best_ddpg = params
    best_neg_sharpe = res['loss']
    print(f"New best sharpe: {best_ddpg}")
    with open("/content/drive/MyDrive/data/portfolio_ddpg_best.txt", "w+") as f:
      json.dump(best_ddpg, f)
      f.write(f"\n\nbest_neg_sharpe: {best_neg_sharpe}")

In [ ]:
import seaborn as sns
def plot_account_values(account_values):
  # account_values['baseline'] = baseline_returns['baseline']
  account_values.reset_index()
  account_values = account_values.set_index('date')

  fig = plt.figure(figsize= (15, 10))
  sns.lineplot(data=account_values)

%matplotlib inline
plot_account_values(df_daily_return)